(numbers)=
# 숫자

## 소개

이 장에서는 숫자 벡터를 만들고 조작하는 데 유용한 도구를 배웁니다. `.count()`에 대해 좀 더 자세히 살펴본 후 다양한 숫자 변환에 대해 자세히 알아볼 것입니다. 그런 다음 다른 유형의 열에 적용할 수 있지만 종종 숫자 열과 함께 사용되는 보다 일반적인 변환에 대해 배웁니다. 그런 다음 몇 가지 더 유용한 요약에 대해 배웁니다.

### 전제 조건

이 장에서는 주로 **pandas**의 함수를 사용하며, 이미 설치되어 있을 가능성이 높지만 터미널에서 `pip install pandas`를 사용하여 설치할 수 있습니다. 가짜 데이터로 만든 장난감 예제뿐만 아니라 nycflights13의 실제 예제를 사용할 것입니다.

먼저 NYC 항공편 데이터를 로드해 보겠습니다.

In [ ]:
# 셀 제거
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

# 플롯 설정
plt.style.use("https://github.com/aeturrell/python4DS/raw/main/plot_style.txt")
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/byuidatascience/data4python4ds/master/data-raw/flights/flights.csv"
flights = pd.read_csv(url)

### 개수

단순히 개수와 약간의 기본 산술만으로 얼마나 많은 데이터 과학을 수행할 수 있는지 놀랍습니다. 따라서 **pandas**는 `.count()` 및 `.value_counts()`를 사용하여 개수를 가능한 한 쉽게 만들려고 노력합니다. 전자는 모든 NA가 아닌 항목의 단순한 개수를 제공합니다.

In [ ]:
flights["dest"].count()

후자는 유형별로 분류된 개수를 제공합니다.

In [ ]:
flights["dest"].value_counts()

이것은 가장 일반적인 범주 순서로 자동 정렬됩니다. `group_by()`, `agg()`를 사용한 다음 count 함수를 사용하여 "수동으로" 동일한 계산을 수행할 수 있습니다. 이렇게 하면 동시에 다른 요약을 계산할 수 있으므로 유용합니다.

In [ ]:
(
    flights.groupby(["dest"])
    .agg(
        mean_delay=("dep_delay", "mean"),
        count_flights=("dest", "count"),
    )
    .sort_values(by="count_flights", ascending=False)
)

가중 개수는 합계일 뿐입니다. 예를 들어 각 비행기가 비행한 마일 수를 "셀" 수 있습니다.

In [ ]:
(flights.groupby("tailnum").agg(miles=("distance", "sum")))

`sum()`과 `isnull()`을 결합하여 결측값을 셀 수 있습니다. 항공편 데이터 세트에서 이것은 취소된 항공편을 나타냅니다. `.isnull()` 다음에 `.sum()`을 적용하는 간단한 문자열 이름이 없으므로(단순히 `sum()`을 실행하는 것과는 달리, 문자열 "sum"으로 지정됨) 아래에서는 람다 함수를 사용해야 합니다.

In [ ]:
(flights.groupby("dest").agg(n_cancelled=("dep_time", lambda x: x.isnull().sum())))

## 숫자 변환

변환 함수의 출력은 입력과 길이가 같습니다. 대부분의 변환 함수는 파이썬에 내장되어 있거나 숫자 패키지 **numpy**와 함께 제공됩니다. 가능한 모든 숫자 변환을 나열하는 것은 비실용적이므로 이 섹션에서는 가장 유용한 변환을 보여줍니다.

기본 숫자 산술은 `+`(더하기), `-`(빼기), `*`(곱하기), `/`(나누기), `**`(거듭제곱), `%`(모듈로) 및 `@`(텐서 곱)으로 수행됩니다. 이러한 함수 대부분은 이미 익숙하므로 많은 설명이 필요하지 않습니다(필요할 때 다른 함수를 찾아볼 수 있음).

길이가 같은 두 개의 숫자 열이 있고 이를 더하거나 빼면 어떤 일이 일어날지 매우 분명합니다. 하지만 열만큼 길지 *않은* 변수가 관련된 경우 어떻게 되는지 이야기해야 합니다. 이는 `flights.assign(air_time = air_time / 60)`과 같은 작업에 중요합니다. `/`의 왼쪽에 336,776개의 숫자가 있지만 오른쪽에는 하나만 있기 때문입니다. 이 경우 **pandas**는 모든 비행 시간을 60으로 나누고 싶다는 것을 이해합니다. 이것을 때때로 '브로드캐스팅'이라고 합니다. 아래는 무슨 일이 일어나고 있는지 설명하려는 다이어그램입니다.

![](https://numpy.org/doc/stable/_images/broadcasting_1.png)

[**numpy** 설명서의 브로드캐스팅](https://numpy.org/doc/stable/user/basics.broadcasting.html)에서 훨씬 더 많은 정보를 찾을 수 있습니다. **pandas**는 **numpy** 위에 구축되었으며 일부 기능을 상속합니다.


두 열에서 작업할 때 **pandas**는 해당 모양을 요소별로 비교합니다. 두 열은 같거나 그중 하나가 스칼라인 경우 호환됩니다. 이러한 조건이 충족되지 않으면 오류가 발생합니다.



### 최소값 및 최대값

산술 함수는 예상대로 작동합니다.

In [ ]:
flights["distance"].max()

때로는 행 *또는* 열 전체에서 최대값 또는 최소값을 보고 싶을 때가 있습니다. **pandas**의 경우와 마찬가지로 `axis=0`(인덱스) 또는 `axis=1`(열)을 해당 함수에 전달하여 함수를 적용할 행 또는 열을 지정할 수 있습니다. 축 지정은 혼란스러울 수 있습니다. 다른 차원을 남겨두고 집계하려는 차원을 묻는다는 것을 기억하십시오. 따라서 각 행에서 최소값을 찾으려면 열을 집계/축소하므로 `axis=1`을 전달해야 합니다.

In [ ]:
df = pd.DataFrame({"x": [1, 5, 7], "y": [3, 2, pd.NA]})
df

이제 행별 최소값을 찾아보겠습니다.

In [ ]:
df.min(axis=1)

### 모듈러 연산

모듈러 연산은 정수에 대해 수행하는 수학 유형의 기술적인 이름입니다. 즉, 정수와 나머지를 생성하는 나눗셈입니다. 파이썬에서 `//`는 정수 나눗셈을 수행하고 `%`는 나머지를 계산합니다.

In [ ]:
print([x for x in range(1, 11)])
print("3으로 나누면 다음과 같습니다.")
print("나머지:")
print([x % 3 for x in range(1, 11)])
print("나눗셈:")
print([x // 3 for x in range(1, 11)])

모듈러 산술은 항공편 데이터 세트에 유용합니다. `sched_dep_time` 변수를 `hour`와 `minute`으로 풀 수 있기 때문입니다.

In [ ]:
flights.assign(
    hour=lambda x: x["sched_dep_time"] // 100,
    minute=lambda x: x["sched_dep_time"] % 100,
)

### 로그

로그는 여러 자릿수에 걸쳐 있는 데이터를 처리하는 데 매우 유용한 변환입니다. 또한 지수 성장을 선형 성장으로 변환합니다. 예를 들어 복리 이자를 생각해 보십시오. `year + 1`에 있는 돈의 양은 `year`에 있던 돈의 양에 이자율을 곱한 값입니다. 그러면 `money = starting * interest ** year`와 같은 공식이 나옵니다.

In [ ]:
import numpy as np

starting = 100
interest = 1.05
money = pd.DataFrame(
    {"year": 2000 + np.arange(1, 51), "money": starting * interest ** np.arange(1, 51)}
)
money.head()

이 데이터를 플로팅하면 지수 곡선이 나타납니다.

In [ ]:
money.plot(x="year", y="money");

y축을 로그 변환하면 직선이 됩니다.

In [ ]:
money.plot(x="year", y="money", logy=True);

이것은 `log(money) = log(starting) + n * log(interest)`가 직선 패턴인 `y = m * x + b`와 일치하기 때문에 직선입니다. 이것은 유용한 패턴입니다. y축을 로그 변환한 후 (대략적으로) 직선을 보면 기본 지수 성장이 있음을 알 수 있습니다.

데이터를 로그 변환하는 경우 **numpy**에서 제공하는 많은 로그 중에서 선택할 수 있지만 일반적으로 사용할 세 가지 로그가 있습니다. `import numpy as np`를 사용하여 가져왔다고 가정하면 `np.log()`(자연 로그, 밑 e), `np.log2()`(밑 2) 및 `np.log10()`(밑 10)이 있습니다.

`log()`의 역함수는 `np.exp()`입니다. `np.log2()` 또는 `np.log10()`의 역함수를 계산하려면 `2**` 또는 `10**`을 사용해야 합니다.

### 반올림

특정 소수 자릿수로 반올림하려면 `.round(n)`을 사용합니다. 여기서 `n`은 반올림하려는 소수 자릿수입니다.

In [ ]:
money.head().round(2)

이것은 개별 열 또는 사전을 통해 열에 차등적으로 적용할 수도 있습니다.

In [ ]:
money.head().round({"year": 0, "money": 1})

`.round(n)`은 가장 가까운 `10**(-n)`으로 반올림하므로 `n = 2`는 가장 가까운 0.01로 반올림합니다. 이 정의는 `.round(-2)`가 가장 가까운 백으로 반올림한다는 것을 의미하므로 유용하며 실제로 그렇게 합니다.

In [ ]:
money.tail().round({"year": 0, "money": -2})

때로는 소수 자릿수 대신 유효 숫자에 따라 반올림하고 싶을 때가 있습니다. 이를 수행하는 정말 쉬운 방법은 없지만 사용자 정의 함수를 정의하여 수행할 수 있습니다. 다음은 유효 숫자 2자리로 반올림하는 예입니다(아래의 `2`를 변경하여 다른 유효 숫자로 반올림).

In [ ]:
money["money"].head().apply(lambda x: float(f'{float(f"{x:.2g}"):g}'))

데이터 프레임 외부에 숫자 배열이나 목록이 있는 경우 **numpy** 함수를 사용할 수 있습니다.

In [ ]:
np.round([1.5, 2.5, 1.4])

**numpy**에는 `.floor()` 및 `.ceil()` 메서드도 있습니다.

In [ ]:
real_nums = 100 * np.random.random(size=10)
real_nums

In [ ]:
np.ceil(real_nums)

In [ ]:
np.floor(real_nums)

**pandas** 데이터 프레임 열에 다음과 같이 항상 **numpy** 함수를 적용할 수 있다는 것을 기억하십시오.

In [ ]:
money["money"].head().apply(np.ceil)

### 누적 및 롤링 집계

**pandas**에는 `.cumsum()`, `.cummax()` 및 `.cummin()`, `.cumprod()`를 포함한 여러 누적 함수가 있습니다.

In [ ]:
money["money"].tail().cumsum()

항상 그렇듯이 `axis=1`을 전달하여 행 전체에도 적용할 수 있습니다.

## 일반 변환

다음 섹션에서는 종종 숫자 벡터와 함께 사용되지만 다른 모든 열 유형에도 적용할 수 있는 몇 가지 일반적인 변환에 대해 설명합니다.

### 순위 매기기

**pandas**의 순위 함수는 `.rank()`입니다. 이전에 만든 데이터를 다시 살펴보고 순위를 매겨 보겠습니다.

In [ ]:
df

In [ ]:
df.rank()

물론 항목이 이미 순위가 매겨져 있으므로 여기에는 변경 사항이 없습니다! 키워드 인수 `pct=True`를 전달하여 백분율 순위를 매길 수도 있습니다.

In [ ]:
df.rank(pct=True)

### 오프셋 및 이동

오프셋을 사용하면 열을 원래 위치를 기준으로 위아래로 '롤링'할 수 있습니다. 즉, 인덱스를 기준으로 위치를 오프셋할 수 있습니다. 이 작업을 수행하는 함수는 `shift()`라고 하며 각각 양수 또는 음수 값을 사용하는지에 따라 선행 또는 후행을 생성합니다. 선행은 인덱스를 기준으로 플로팅할 때 데이터의 패턴을 왼쪽(즉, 시간적으로 더 일찍)으로 이동시키고 후행은 패턴을 시간적으로 더 늦게(즉, 오른쪽으로) 이동시킨다는 것을 기억하십시오. 선행 및 후행은 시계열 데이터에 특히 유용합니다(아직 보지 못함).

이전의 `money` 데이터 프레임을 사용하여 오프셋 예를 살펴보겠습니다.

In [ ]:
money["money_lag_5"] = money["money"].shift(5)
money["money_lead_10"] = money["money"].shift(-10)
money.set_index("year").plot();

### 연습

1.  순위 함수를 사용하여 가장 지연된 항공편 10개를 찾습니다.

2.  어떤 비행기(`"tailnum"`)가 최악의 정시 기록을 가지고 있습니까?

3.  가능한 한 지연을 피하려면 하루 중 언제 비행해야 합니까?

4.  각 목적지에 대해 총 지연 시간을 계산합니다. 각 항공편에 대해 해당 목적지의 총 지연 시간 비율을 계산합니다.

5.  지연은 일반적으로 시간적으로 상관 관계가 있습니다. 초기 지연을 유발한 문제가 해결된 후에도 이전 항공편이 출발할 수 있도록 이후 항공편이 지연됩니다. `.shift()`를 사용하여 한 시간의 평균 항공편 지연이 이전 시간의 평균 지연과 어떻게 관련되는지 탐색합니다.

6.  각 목적지를 살펴보십시오. 의심스러울 정도로 빠른 항공편을 찾을 수 있습니까? (즉, 잠재적인 데이터 입력 오류를 나타내는 항공편). 해당 목적지까지 가장 짧은 항공편을 기준으로 항공편의 비행 시간을 계산합니다. 어떤 항공편이 비행 중 가장 많이 지연되었습니까?

7.  최소 두 개의 항공사가 운항하는 모든 목적지를 찾습니다. 해당 목적지를 사용하여 동일한 목적지에 대한 성능을 기준으로 항공사의 상대적 순위를 매깁니다.


## 더 유용한 요약 통계

분석에 `.mean()`, `.count()` 및 `.value_counts()`가 얼마나 유용한지 살펴보았습니다. 그러나 **pandas**에는 훨씬 더 많은 내장 요약 통계 함수가 있습니다. 여기에는 `.median()`(항공편 데이터의 시간별 출발 지연을 볼 때 평균과 중앙값을 비교하는 것이 흥미로울 수 있음), `.mode()`, `.min()` 및 `.max()`가 포함됩니다.

유용한 요약 통계 클래스는 `.quantile` 함수에서 제공되며, 이는 `.quantile(0.5)`의 경우 `median`과 동일합니다. x%의 분위수는 x%의 값이 그보다 낮은 값입니다. (이 정의에 따라 `.quantile(1)`은 `.max()`와 동일합니다.) 25번째 백분위수를 예로 들어 보겠습니다.

In [ ]:
money["money"].quantile(0.25)

때로는 하나의 백분위수만 원하는 것이 아니라 여러 개를 원할 때가 있습니다. **pandas**는 분위수 목록을 전달할 수 있도록 하여 이를 매우 쉽게 만듭니다.

In [ ]:
money["money"].quantile([0, 0.25, 0.5, 0.75])

### 산포도

때로는 데이터의 대부분이 어디에 있는지보다 데이터가 어떻게 퍼져 있는지에 더 관심이 있을 수 있습니다.
일반적으로 사용되는 두 가지 요약은 표준 편차 `.std()`와 사분위수 범위이며, 관련 분위수에서 계산할 수 있습니다. 즉, 75% 분위수에서 25% 분위수를 뺀 값이며 데이터의 중간 50%가 포함된 범위를 제공합니다.

이를 사용하여 항공편 데이터의 작은 이상한 점을 밝힐 수 있습니다. 공항은 항상 같은 장소에 있으므로 출발지와 목적지 간의 거리 확산이 0일 것으로 예상할 수 있습니다. 그러나 아래 코드는 한 공항인 [EGE](https://en.wikipedia.org/wiki/Eagle_County_Regional_Airport)가 이동했을 수 있음을 보여줍니다.

In [ ]:
(
    flights.groupby(["origin", "dest"])
    .agg(
        distance_sd=("distance", lambda x: x.quantile(0.75) - x.quantile(0.25)),
        count=("distance", "count"),
    )
    .query("distance_sd > 0")
)

### 분포

위에서 설명한 모든 요약 통계는 분포를 단일 숫자로 줄이는 방법이라는 것을 기억하는 것이 중요합니다. 즉, 근본적으로 축소적이며 잘못된 요약을 선택하면 그룹 간의 중요한 차이를 쉽게 놓칠 수 있습니다. 그렇기 때문에 집계 통계를 사용하는 것 외에도 항상 값의 분포를 시각화하는 것이 좋습니다.

아래 플롯은 출발 지연의 전체 분포를 보여줍니다. 분포가 너무 왜곡되어 데이터의 대부분을 보려면 확대해야 합니다. 이는 평균이 좋은 요약이 아닐 가능성이 높으며 대신 중앙값을 선호할 수 있음을 시사합니다.

In [ ]:
flights["dep_delay"].plot.hist(bins=50, title="         분포: 지연 시간");

In [ ]:
flights.query("dep_delay <= 120")["dep_delay"].plot.hist(
    bins=50, title="         분포: 지연 시간"
);

`"dep_delay"`의 두 히스토그램. 전자의 경우 0 주변에 매우 큰 스파이크가 있고 막대 높이가 급격히 감소하며 플롯 대부분에서 막대가 너무 짧아 볼 수 없다는 점을 제외하고는 패턴을 보기 매우 어렵습니다. 후자의 경우 2시간보다 큰 지연을 버렸는데 스파이크가 0보다 약간 아래에서 발생하지만(즉, 대부분의 항공편이 몇 분 일찍 출발함) 그 후에도 여전히 매우 가파른 감소가 있음을 알 수 있습니다.

작업 중인 데이터에 맞게 특별히 조정된 사용자 지정 요약을 탐색하는 것을 두려워하지 마십시오. 이 경우 일찍 출발한 항공편과 늦게 출발한 항공편을 별도로 요약하거나 값이 너무 왜곡되어 있으므로 로그 변환을 시도할 수 있습니다. 마지막으로 요약을 만들 때 각 그룹의 관찰 수를 포함하는 것이 항상 좋은 생각이라는 것을 잊지 마십시오.

### 연습

1.  항공편 그룹의 일반적인 지연 특성을 평가하는 5가지 이상의 다양한 방법을 브레인스토밍합니다.
    다음 시나리오를 고려하십시오.

    -   항공편이 50%의 확률로 15분 일찍 도착하고 50%의 확률로 15분 늦게 도착합니다.
    -   항공편이 항상 10분 늦게 도착합니다.
    -   항공편이 50%의 확률로 30분 일찍 도착하고 50%의 확률로 30분 늦게 도착합니다.
    -   99%의 경우 항공편이 정시에 도착합니다. 1%의 경우 2시간 늦게 도착합니다.

    도착 지연과 출발 지연 중 어느 것이 더 중요하다고 생각하십니까?

2.  어떤 목적지에서 풍속 변화가 가장 큽니까?

3.  EGE의 모험을 더 자세히 탐색하기 위해 플롯을 만듭니다.
    공항 위치가 변경되었다는 증거를 찾을 수 있습니까?